In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"
rng = np.random.default_rng(42)

def mae(y, yhat):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    return np.mean(np.abs(y - yhat))

def rmse(y, yhat):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    return np.sqrt(np.mean((y - yhat) ** 2))

def mape(y, yhat):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    mask = y != 0
    if not mask.any():
        return np.nan
    return np.mean(np.abs((y[mask] - yhat[mask]) / y[mask])) * 100

def smape(y, yhat):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    denom = np.abs(y) + np.abs(yhat)
    mask = denom != 0
    if not mask.any():
        return np.nan
    return np.mean(2 * np.abs(y[mask] - yhat[mask]) / denom[mask]) * 100

def wape(y, yhat):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    denom = np.sum(np.abs(y))
    if denom == 0:
        return np.nan
    return np.sum(np.abs(y - yhat)) / denom * 100

def mase(y, yhat, y_train):
    y = np.asarray(y)
    yhat = np.asarray(yhat)
    y_train = np.asarray(y_train)
    naive_denom = np.mean(np.abs(np.diff(y_train)))
    if naive_denom == 0:
        return np.nan
    return mae(y, yhat) / naive_denom


In [2]:
n = 120
t = np.arange(n)
trend = 0.2 * t
season = 10 * np.sin(2 * np.pi * t / 12)
noise = rng.normal(0, 3, size=n)
y = 50 + trend + season + noise

index = pd.period_range('2015-01', periods=n, freq='M').to_timestamp()
series = pd.Series(y, index=index, name='y')

train = series.iloc[:-24]
test = series.iloc[-24:]


In [3]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train, mode='lines', name='Train'))
fig.add_trace(go.Scatter(x=test.index, y=test, mode='lines', name='Test', line=dict(color='black')))
fig.add_vline(x=test.index[0], line_dash='dash', line_color='gray')
fig.update_layout(title='Synthetic series with train/test split', xaxis_title='Date', yaxis_title='y')
fig.show()


In [4]:
def forecast_naive(train, horizon_index):
    return pd.Series([train.iloc[-1]] * len(horizon_index), index=horizon_index, name='Naive')

def forecast_seasonal_naive(train, horizon_index, season_length=12):
    last_season = train.iloc[-season_length:]
    reps = int(np.ceil(len(horizon_index) / season_length))
    vals = np.tile(last_season.values, reps)[: len(horizon_index)]
    return pd.Series(vals, index=horizon_index, name=f'Seasonal naive (s={season_length})')

def forecast_moving_average(train, horizon_index, window=6):
    mean_val = train.iloc[-window:].mean()
    return pd.Series([mean_val] * len(horizon_index), index=horizon_index, name=f'Moving average (w={window})')

pred_naive = forecast_naive(train, test.index)
pred_seasonal = forecast_seasonal_naive(train, test.index, season_length=12)
pred_ma = forecast_moving_average(train, test.index, window=6)


In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train, mode='lines', name='Train', line=dict(color='gray')))
fig.add_trace(go.Scatter(x=test.index, y=test, mode='lines', name='Test', line=dict(color='black')))
fig.add_trace(go.Scatter(x=test.index, y=pred_naive, mode='lines', name=pred_naive.name))
fig.add_trace(go.Scatter(x=test.index, y=pred_seasonal, mode='lines', name=pred_seasonal.name))
fig.add_trace(go.Scatter(x=test.index, y=pred_ma, mode='lines', name=pred_ma.name))
fig.add_vline(x=test.index[0], line_dash='dash', line_color='gray')
fig.update_layout(title='Baseline forecasts on the test horizon', xaxis_title='Date', yaxis_title='y')
fig.show()


In [6]:
def compute_metrics(y_true, y_pred, y_train):
    return {
        'MAE': mae(y_true, y_pred),
        'RMSE': rmse(y_true, y_pred),
        'MAPE (%)': mape(y_true, y_pred),
        'sMAPE (%)': smape(y_true, y_pred),
        'WAPE (%)': wape(y_true, y_pred),
        'MASE': mase(y_true, y_pred, y_train),
    }

metrics = pd.DataFrame({
    pred_naive.name: compute_metrics(test.values, pred_naive.values, train.values),
    pred_seasonal.name: compute_metrics(test.values, pred_seasonal.values, train.values),
    pred_ma.name: compute_metrics(test.values, pred_ma.values, train.values),
}).T

metrics.round(3)


,MAE,RMSE,MAPE (%),sMAPE (%),WAPE (%),MASE
Naive,11.237,13.163,15.054,16.710,15.856,2.765
Seasonal naive (s=12),4.294,5.526,6.092,6.412,6.059,1.056
Moving average (w=6),10.360,12.251,13.857,15.267,14.618,2.549


In [7]:
scale_dependent = ['MAE', 'RMSE']
scale_free = ['MAPE (%)', 'sMAPE (%)', 'WAPE (%)', 'MASE']

metrics_long = metrics.reset_index().rename(columns={'index': 'model'})

fig1 = px.bar(
    metrics_long.melt(id_vars='model', value_vars=scale_dependent),
    x='model',
    y='value',
    color='variable',
    barmode='group',
    title='Scale-dependent metrics (lower is better)',
)
fig1.update_layout(xaxis_title='Model', yaxis_title='Metric value')
fig1.show()

fig2 = px.bar(
    metrics_long.melt(id_vars='model', value_vars=scale_free),
    x='model',
    y='value',
    color='variable',
    barmode='group',
    title='Scale-free metrics (lower is better)',
)
fig2.update_layout(xaxis_title='Model', yaxis_title='Metric value')
fig2.show()


In [8]:
scale = 10
scaled = pd.DataFrame({
    'Original': compute_metrics(test.values, pred_seasonal.values, train.values),
    'Scaled x10': compute_metrics(test.values * scale, pred_seasonal.values * scale, train.values * scale),
}).T

scaled[['MAE', 'RMSE', 'MAPE (%)', 'sMAPE (%)', 'WAPE (%)', 'MASE']].round(3)


,MAE,RMSE,MAPE (%),sMAPE (%),WAPE (%),MASE
Original,4.294,5.526,6.092,6.412,6.059,1.056
Scaled x10,42.938,55.262,6.092,6.412,6.059,1.056


In [9]:
test_outlier = test.copy()
test_outlier.iloc[-1] = test_outlier.iloc[-1] + 30

outlier_effect = pd.DataFrame({
    'Baseline': compute_metrics(test.values, pred_seasonal.values, train.values),
    'With outlier': compute_metrics(test_outlier.values, pred_seasonal.values, train.values),
}).T

outlier_effect[['MAE', 'RMSE', 'MAPE (%)', 'sMAPE (%)', 'WAPE (%)', 'MASE']].round(3)


,MAE,RMSE,MAPE (%),sMAPE (%),WAPE (%),MASE
Baseline,4.294,5.526,6.092,6.412,6.059,1.056
With outlier,5.544,9.439,7.208,7.893,7.687,1.364
